In [ ]:
import specpy

im = specpy.get_application()
ms = im.active_measurement()

In [ ]:
# simple parameter setting example

ypos = ms.parameters('ExpControl/scan/range/coarse_y/g_off')
ypos += 1000e-06
ms.set_parameters('ExpControl/scan/range/coarse_y/g_off', ypos)

In [ ]:
im.run(ms)

Test running pipeline with current measurement settings

In [ ]:
import h5py as h5

from pipeline2.taskgeneration import AcquisitionTaskGenerator
from pipeline2.taskgeneration.taskgeneration import TimeSeriesCallback
import pipeline2 as p2


save_path = 'D:/AUTOMATION/TEST/'

# make output dir if it does not exist already
if not os.path.exists(save_path):
    os.makedirs(save_path)

pll = p2.PipelineLevels('image')
name_handler = p2.DefaultNameHandler(save_path, pll)

# get all parameters of active measurement
params = specpy.get_application().active_measurement().active_configuration().parameters('')

# task generator to image with current settings
task_gen_current_settings = AcquisitionTaskGenerator(pll.image,
                                 p2.taskgeneration.JSONFileConfigLoader([params])
)

# abuse name_handler to get h5 filename
# calling with empty index gives just the random hash
h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

# open resulting h5 file in context manager
with h5.File(h5file_name, mode='a') as h5fd:
    data_store = p2.HDF5DataStore(h5fd, pll)

    # init pipeline
    pl = (p2.AcquisitionPipeline('test-pipeline')
            .withImspectorConnection(p2.imspector.ImspectorConnection(im).withVerbose())
            .withPipelineLevels(pll)
            .withNameHandler(name_handler)
        )

    # attach h5 data store
    pl.withDataStorage(data_store)

    task_gen_current_settings(pl)

    pl.run()


Test timeseries with current settings

In [ ]:
import h5py as h5

from pipeline2.taskgeneration import AcquisitionTaskGenerator
from pipeline2.taskgeneration.taskgeneration import TimeSeriesCallback
import pipeline2 as p2


save_path = 'D:/AUTOMATION/TEST/'

# make output dir if it does not exist already
if not os.path.exists(save_path):
    os.makedirs(save_path)

pll = p2.PipelineLevels('timepoint', 'image')
name_handler = p2.DefaultNameHandler(save_path, pll)

# get all parameters of active measurement
params = specpy.get_application().active_measurement().active_configuration().parameters('')

# task generator to image with current settings
task_gen_current_settings = AcquisitionTaskGenerator(pll.image,
                                 p2.taskgeneration.JSONFileConfigLoader([params])
)

timeseries_callback = TimeSeriesCallback(pll.timepoint)
timeseries_callback.time_points = [0, 30, 60]

# abuse name_handler to get h5 filename
# calling with empty index gives just the random hash
h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

# open resulting h5 file in context manager
with h5.File(h5file_name, mode='a') as h5fd:
    data_store = p2.HDF5DataStore(h5fd, pll)

    # init pipeline
    pl = (p2.AcquisitionPipeline('test-pipeline')
            .withImspectorConnection(p2.imspector.ImspectorConnection(im).withVerbose())
            .withPipelineLevels(pll)
            .withNameHandler(name_handler)
        )

    # attach h5 data store
    pl.withDataStorage(data_store)

    pl.withCallbackAtLevel(timeseries_callback, pll.timepoint)
    pl.withCallbackAtLevel(task_gen_current_settings, pll.timepoint)

    timeseries_callback.initialize_time_series(pl)
    pl.run()


